In [30]:
from requests import get
from scipy.stats import ttest_rel

metric = 'cov'
cutoff = '10'
n_questions = 10
experiment_subset = 'equal'
significance_level = 0.05
data = get(f'https://mindreader.tech/spectate/results/{experiment_subset}/{metric}/{cutoff}').json()
de_only = False
recommenders = ['ppr-joint', 'knn', 'mf', 'ppr-collab', 'ppr-kg']

if not de_only:
    recommenders = [f'{recommender}-rec' for recommender in recommenders]

interviewer_a = 'greedy-adaptive'
interviewer_b = 'pop'

for recommender in recommenders:
    model_a = f'{interviewer_a}-{recommender}'
    model_b = f'{interviewer_b}-{recommender}'
    
    if model_a not in data or model_b not in data:
        continue
        
    data_a = data[model_a][n_questions - 1]
    data_b = data[model_b][n_questions - 1]
    
    t, p = ttest_rel(data_a, data_b)
    print(f'{"*" if p <= significance_level else ""}{recommender}: {p:.5f}')


ppr-joint-rec: 0.10450
knn-rec: 0.10770
mf-rec: 0.12233
ppr-collab-rec: 0.29573
ppr-kg-rec: 0.35447
